In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [2]:
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from cv2_plt_imshow import cv2_plt_imshow, plt_format

In [3]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog


In [75]:
video = "video.mp4"

cap = cv2.VideoCapture(video)
cnt=0

if not cap.isOpened():
        print("Error opening video stream or file")

In [76]:
ret,first_frame = cap.read()

while(cap.isOpened()):   
    ret, frame = cap.read()  
    if ret == True:       
        cv2.imwrite('frames/'+str(cnt)+'.png', frame)
        cnt=cnt+1
        if(cnt==200):
              break 
    else: 
        break

In [77]:

cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_C4_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_C4_3x.yaml")
predictor = DefaultPredictor(cfg)



WARNING [12/16 16:59:22 d2.modeling.backbone.resnet]: ResNet.make_stage(first_stride=) is deprecated!  Use 'stride_per_block' or 'stride' instead.


In [78]:

def midpoint_person(image,person,index):
    x1,y1,x2,y2 = person[index]
    _ = cv2.rectangle(image, (x1, y1), (x2, y2), (255,0,0), 2)
    
    x_mid = int((x1+x2)/2)
    y_mid = int(y2)
    mid   = (x_mid,y_mid)

    _ = cv2.circle(image, mid, 5, (0, 0, 255), -1)
    cv2.putText(image, str(index), mid, cv2.FONT_HERSHEY_SIMPLEX,1, (255, 255, 255), 2, cv2.LINE_AA)
  
    return mid

In [79]:
from scipy.spatial import distance

def calculate_distance(midpoint,person):
    dist = np.zeros((person,person))
    for i in range(person):
        for j in range(i+1,person):
              if i!=j:
                dst = distance.euclidean(midpoint[i], midpoint[j])
                dist[i][j]=dst
    return dist

In [80]:
def closest_people(dist,num_person,thres):
    p1=[]
    p2=[]
    d=[]
    for i in range(num_person):
        for j in range(i,num_person):
              if( (i!=j) & (dist[i][j]<=thres)):
                p1.append(i)
                p2.append(j)
                d.append(dist[i][j])
    return p1,p2,d

In [81]:
def turn_red(image,person,p1,p2):
    risky = np.unique(p1+p2)
    for i in risky:
        x1,y1,x2,y2 = person[i]
        _ = cv2.rectangle(image, (x1, y1), (x2, y2), (0,0,255), 2)  
    return image

In [82]:
import os
import re

names=os.listdir('frames/')
names.sort(key=lambda f: int(re.sub('\D', '', f)))

In [83]:
def safe_distance(image_name,thres):

    image = cv2.imread('frames/'+image_name)
    outputs = predictor(image)
    classes=outputs['instances'].pred_classes.cpu().numpy()    
    bbox=outputs['instances'].pred_boxes.tensor.cpu().numpy()    
    index = np.where(classes==0)[0]
    person=bbox[index]
    midpoints = [midpoint_person(image,person,i) for i in range(len(person))]
    num_person = len(midpoints)
    dist= calculate_distance(midpoints,num_person)
    p1,p2,d= closest_people(dist,num_person,thres)
    image = turn_red(image,person,p1,p2)
    cv2.imwrite('frames/'+image_name,image)
    return 0

In [84]:
thres =100
_ = [safe_distance(names[i],thres) for i in range(len(names))]

In [85]:
frames = os.listdir('frames/')
frames.sort(key=lambda f: int(re.sub('\D', '', f)))

frame_array=[]

for i in range(len(frames)):
    
    image = cv2.imread('frames/'+frames[i])
    height, width, layers = image.shape
    size = (width,height)
    
    frame_array.append(image)

out = cv2.VideoWriter('safe_output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
 
for i in range(len(frame_array)):
    out.write(frame_array[i])
out.release()